In [85]:
import csv
from rdflib import Graph, URIRef, Literal, Namespace, RDF, XSD
from datetime import datetime
from rdflib.plugins.sparql import prepareQuery
import random
import uuid

# Créez un espace de nom pour votre ontologie
kesaio = Namespace("http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/")
common_core_ns = Namespace("http://www.ontologyrepository.com/CommonCoreOntologies/")
hasRole = URIRef("http://purl.obolibrary.org/obo/RO_0000087")
participatesIn = URIRef("http://purl.obolibrary.org/obo/RO_0000056")

In [86]:
# Ouvrez le fichier CSV d'entrée
with open("dataFromWikidata.csv", "r", newline="", encoding="utf-8") as infile:
    reader = csv.DictReader(infile)
    
    # Créez un dictionnaire pour stocker les enregistrements uniques
    unique_records = {}
    
    for row in reader:
        # Remplacez les espaces par des underscores dans les champs spécifiques
        row["seismeLabel"] = row["seismeLabel"].replace(" ", "_")
        row["lieuLabel"] = row["lieuLabel"].replace(" ", "_")
        row["paysLabel"] = row["paysLabel"].replace(" ", "_")
        row["description"] = row["description"].replace(" ", "_")

        # Obtenez les valeurs clés pour identifier les doublons
        key = (row["seismeLabel"])
        
        # Si la clé n'existe pas dans le dictionnaire, ajoutez l'enregistrement
        if key not in unique_records:
            unique_records[key] = row

# Créez un nouveau fichier CSV pour les enregistrements uniques
with open("dataFromWikidataNew.csv", "w", newline="", encoding="utf-8") as outfile:
    fieldnames = reader.fieldnames  # Utilisez les mêmes en-têtes de colonnes que le fichier d'origine
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    # Écrivez les en-têtes de colonnes dans le fichier de résultat
    writer.writeheader()
    
    # Écrivez les enregistrements uniques dans le fichier de résultat
    for record in unique_records.values():
        writer.writerow(record)

In [87]:
def sévérité(kesaio, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, seisme_uri, nombre_morts):
    if nombre_morts in severite_niveau_0:
        severite = "Niveau 0"
    elif nombre_morts in severite_niveau_1:
        severite = "Niveau 1"
    elif nombre_morts in severite_niveau_2:
        severite = "Niveau 2"
    else:
        severite = "Niveau inconnu"

        # Ajoutez une propriété de sévérité à l'instance de catastrophe
    g.add((seisme_uri, kesaio["sévérite_de_la_catastrophe"], Literal(severite, datatype=XSD.string)))

In [88]:
# Ouvrez le fichier CSV contenant les données
def creer_victimes(kesaio, common_core_ns, g, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, row, seisme_uri):
    l = [ua_uri, ur_uri, ump_uri, ind_uri]
    for i in range(0, int(row["nombreMorts"])):  # Créez 25 victimes
            # Créez une instance unique de la classe "Etat_victime"
        etat_victime_id = kesaio["Etat_victime_Mort" + str(i)]  # Par exemple, "Etat_victime_1", "Etat_victime_2", ...
            
            # Utilisez l'instance "DCD" pour attribuer l'état de décédé à chaque instance d'état de victime
        g.add((etat_victime_id, RDF.type, kesaio["Etat_victime"]))  # Assurez-vous que la classe "Etat_victime" est définie dans votre ontologie
        g.add((etat_victime_id, common_core_ns["designated_by"], dcd_uri))  # Reliez l'état de victime à l'instance "DCD"

            # Créez une instance unique de la classe "Victime" avec un identifiant unique (Vi)
        victime_id = kesaio["Mort" + str(i)]  # Par exemple, "Vi1", "Vi2", ...
            
            # Ajoutez le triple RDF pour chaque instance de victime
        g.add((victime_id, RDF.type, kesaio["Victime"]))  # Assurez-vous que la classe "Victime" est définie dans votre ontologie
        g.add((victime_id, kesaio["has_status"], etat_victime_id))
        g.add((seisme_uri, kesaio["engendre"], victime_id))


    for i in range(0, int(row["nombreBlesse"])):  # Créez 25 victimes
            # Créez une instance unique de la classe "Etat_victime"
        etat_victime_id = kesaio["Etat_victime_Blessé" + str(i)]  # Par exemple, "Etat_victime_1", "Etat_victime_2", ...
            
        element_aleatoire = random.choice(l)
            # Utilisez l'instance "DCD" pour attribuer l'état de décédé à chaque instance d'état de victime
        g.add((etat_victime_id, RDF.type, kesaio["Etat_victime"]))  # Assurez-vous que la classe "Etat_victime" est définie dans votre ontologie
        g.add((etat_victime_id, common_core_ns["designated_by"], element_aleatoire))  # Reliez l'état de victime à l'instance "DCD"

            # Créez une instance unique de la classe "Victime" avec un identifiant unique (Vi)
        victime_id = kesaio["Blessé" + str(i)]  # Par exemple, "Vi1", "Vi2", ...
            
            # Ajoutez le triple RDF pour chaque instance de victime
        g.add((victime_id, RDF.type, kesaio["Victime"]))  # Assurez-vous que la classe "Victime" est définie dans votre ontologie
        g.add((victime_id, kesaio["has_status"], etat_victime_id))
        g.add((seisme_uri, kesaio["engendre"], victime_id))

In [89]:
def catastrophe(kesaio, common_core_ns, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, creer_victimes, sévérité, row):
    date_uri = kesaio[row["date"]]
        
    # Ajoutez des triples RDF pour la classe Date
    g.add((date_uri, RDF.type, kesaio["Date"]))
        
    # Extraitz le jour, le mois et l'année du format de date "2016-02-06T00:00:00Z"
    date_str = row["date"]
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
    jour = date_obj.day
    mois = date_obj.month
    annee = date_obj.year
        
    # Créez des ressources URI pour les composants jour, mois et année
    jour_uri = kesaio[f"jour_{jour}"]
    mois_uri = kesaio[f"mois_{mois}"]
    annee_uri = kesaio[f"{annee}"]

    # Ajoutez des triples RDF pour les composants jour, mois et année
    g.add((jour_uri, RDF.type, common_core_ns["GregorianDay"]))
    g.add((mois_uri, RDF.type, kesaio["Gregorian_Month"]))
    g.add((annee_uri, RDF.type, common_core_ns["GregorianYear"]))
        
    # Liez les composants jour, mois et année à l'instance de date
    g.add((date_uri, kesaio["has_day"], jour_uri))
    g.add((date_uri, kesaio["has_month"], mois_uri))
    g.add((date_uri, kesaio["has_year"], annee_uri))

    seisme_uri = kesaio[row["seismeLabel"]]
    pays_uri = kesaio[row["paysLabel"]]
    city_uri = kesaio[row["lieuLabel"]]

        
    # Ajoutez des triples RDF en utilisant l'espace de nom
    g.add((seisme_uri, RDF.type, kesaio["Séisme"]))
    g.add((pays_uri, RDF.type, common_core_ns["State"]))
    g.add((city_uri, RDF.type, common_core_ns["City"]))
        
    # Ajoutez des triples RDF pour les dataproperties
    g.add((seisme_uri, kesaio["nombre_de_morts"], Literal(row["nombreMorts"], datatype=XSD.integer)))
    g.add((seisme_uri, kesaio["nombre_de_blessés"], Literal(row["nombreBlesse"], datatype=XSD.integer)))
    g.add((seisme_uri, kesaio["description"], Literal(row["description"], datatype=XSD.string)))
        
    # Ajoutez des triples RDF pour les relations entre les instances
    g.add((seisme_uri, common_core_ns["occurs_at"], city_uri))
    g.add((city_uri, common_core_ns["spatial_part_of"], pays_uri))
    g.add((seisme_uri, common_core_ns["occurs_on"], date_uri))

    # Obtenez le nombre de morts à partir du CSV (assurez-vous que le champ est converti en entier)
    nombre_morts = int(row["nombreMorts"])
        
    # Déterminez la sévérité en fonction du nombre de morts
    sévérité(kesaio, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, seisme_uri, nombre_morts)

    creer_victimes(kesaio, common_core_ns, g, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, row, seisme_uri)
    # Créez une instance unique de la classe "Sapeur_pompier" avec un identifiant unique (Sapeur_pompier_i)

    act_uri, uri_op = kesaio["Acte_de_réponse_"], kesaio["Opération_"]
    g.add((act_uri, RDF.type, kesaio["Acte_de_réponse"]))
    g.add((uri_op, RDF.type, kesaio["Opération"]))
    g.add((act_uri, common_core_ns["has_process_part"], uri_op))
    g.add((seisme_uri, kesaio["trigger_to"], act_uri))

    return uri_op


In [90]:
# Créez une instance de Graph pour votre ontologie RDF
g = Graph()

# Chargez votre ontologie RDF existante (si nécessaire)
g.parse("file8.rdf")

# Définissez les plages de nombre de morts pour chaque niveau de sévérité
severite_niveau_0 = range(0, 100)
severite_niveau_1 = range(100, 500)
severite_niveau_2 = range(500, 1000000)

dcd_uri = kesaio["décédé"]

ua_uri = kesaio["ua"]
ur_uri = kesaio["ur"]
# ud_uri = kesaio["ud"]
ump_uri = kesaio["ump"]
ind_uri = kesaio["indemne"]

with open("dataFromWikidataNew.csv", "r", newline="", encoding="utf-8") as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        # Créez une ressource URI pour chaque instance de date
        uri_op = catastrophe(kesaio, common_core_ns, g, severite_niveau_0, severite_niveau_1, severite_niveau_2, dcd_uri, ua_uri, ur_uri, ump_uri, ind_uri, creer_victimes, sévérité, row)

In [91]:
# What is the nature of the disaster <X>?
requete_sparql = """
SELECT ?type ?description
WHERE {
    ?seisme rdf:type ?type.
    ?seisme kesaio:description ?description .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenir le nom de la classe en extrayant la partie après le dernier "/"
    classe_nom = resultat.type.split("/")[-1]
    print("Nature:", classe_nom)
    print("Description:", resultat.description)

Nature: Séisme
Description: séisme_au_Japon_en_2018


In [92]:
# When did the disaster <X> take place?
requete_sparql = """
SELECT ?jour ?mois ?annee
WHERE {
    ?seisme cco:occurs_on ?date.
    ?date kesaio:has_day ?jour .
    ?date kesaio:has_month ?mois .
    ?date kesaio:has_year ?annee .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/", 
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    jour = str(resultat.jour.split('/')[-1])
    mois = str(resultat.mois.split('/')[-1])
    annee = str(resultat.annee.split('/')[-1])

    print("Jour:", jour)
    print("Mois:", mois)
    print("Année:", annee)

Jour: jour_6
Mois: mois_9
Année: 2018


In [93]:
# Where did the disaster <X> take place?
requete_sparql = """
SELECT ?city ?pays
WHERE {
    ?seisme cco:occurs_at ?city.
    ?city cco:spatial_part_of ?pays
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    city = str(resultat.city.split('/')[-1])
    pays = str(resultat.pays.split('/')[-1])

    print("Ville:", city)
    print("Pays:", pays)

Ville: Tomakomai
Pays: Japon


In [94]:
# What is the criticality level of the disaster <X>?
requete_sparql = """
SELECT ?sévérité ?blessés ?morts
WHERE {
    ?seisme kesaio:nombre_de_morts ?morts .
    ?seisme kesaio:nombre_de_blessés ?blessés .
    ?seisme kesaio:sévérite_de_la_catastrophe ?sévérité .
}
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/"})

# Exécutez la requête et itérez sur les résultats
resultats = g.query(requete)

for resultat in resultats:
    # Obtenez les valeurs en tant que chaînes de caractères
    sévérité = str(resultat.sévérité.split('/')[-1])
    blessés = str(resultat.blessés.split('/')[-1])
    morts = str(resultat.morts.split('/')[-1])

    print("Sévérité de la catastrophe:", sévérité)
    print("Nombre de blessés:", blessés)
    print("Nombre de morts:", morts)

Sévérité de la catastrophe: Niveau 0
Nombre de blessés: 10
Nombre de morts: 5


# PRV (Point de rassemblement des victimes)

In [95]:
coordonnees = kesaio[row["coordonnees"]]
# Trouvez l'indice du début des coordonnées
start_index = coordonnees.find("(")
# Trouvez l'indice de fin des coordonnées
end_index = coordonnees.find(")")
# Extrayez la partie des coordonnées entre les parenthèses
coord_part = coordonnees[start_index + 1:end_index]
# Séparez les coordonnées en latPRVitude et lonPRVgitude en utilisant l'espace comme séparateur
latPRV, lonPRV = coord_part.split()
# Convertissez les chaînes en nombres flottants si nécessaire
latPRV = float(latPRV) + 50
lonPRV = float(lonPRV) + 50

uri_prv = kesaio["Point_de_rassemblement_des_victimes" + "_1"]
g.add((uri_prv, RDF.type, kesaio["Point_de_rassemblement_des_victimes"]))
g.add((uri_prv, kesaio["longitude"], Literal(lonPRV, datatype=XSD.decimal)))
g.add((uri_prv, kesaio["latitude"], Literal(latPRV, datatype=XSD.decimal)))

uri_offPRV = kesaio["officierRassemblement"]

for i in range(0, 3): 
    identifiant_unique = uuid.uuid4()
    officierPRV_id = kesaio["OfficierPRV" + str(i)]  
    g.add((officierPRV_id, RDF.type, kesaio["Membre_SAMU"])) 
    g.add((officierPRV_id, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((officierPRV_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((officierPRV_id, hasRole, uri_offPRV))
    g.add((uri_offPRV, kesaio["estSuperviséPAr"], kesaio["commandantSAMU"]))
    g.add((officierPRV_id, kesaio["monter"], uri_prv))
    # g.add((uri_prv, kesaio["installed_by"], uri_offPRV))

http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Point(84.725 28.165) does not look like a valid URI, trying to serialize this will break.


# Acte de Ramassage des victimes

In [96]:
uri_actDeRamassage = kesaio["Acte_de_ramassage_des_victimes" + "_1"]
g.add((uri_actDeRamassage, RDF.type, kesaio["Acte_de_ramassage_des_victimes"]))
g.add((uri_actDeRamassage, common_core_ns["occurs_at"], uri_prv))
g.add((uri_op, common_core_ns["has_process_part"], uri_actDeRamassage))

nombreMorts = int(row["nombreMorts"]) 
nombreBlesse = int(row["nombreBlesse"])

nombreDeVictime = nombreMorts + nombreBlesse

In [97]:
nombre_pompiers = 0
nombre_policiers = 0
nombre_officiers_samu = 0

if nombreDeVictime < 8:
    nombre_pompiers = 1
    nombre_policiers = 1
    nombre_officiers_samu = 1
elif 8 <= nombreDeVictime <= 25:
    nombre_pompiers = 5
    nombre_policiers = 2
    nombre_officiers_samu = 5
else:
    nombre_pompiers = 10
    nombre_policiers = 5
    nombre_officiers_samu = 10

In [98]:
for i in range(0,nombre_pompiers): 
    pompier_id = kesaio["Pompier" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((pompier_id, RDF.type, kesaio["Pompier"])) 
    g.add((pompier_id, hasRole, kesaio["equipierMedical"]))
    g.add((pompier_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((pompier_id, common_core_ns["is_affiliated_with"], kesaio["sapeurPompier"]))
    g.add((pompier_id, participatesIn, uri_actDeRamassage))
for i in range(0,nombre_policiers): 
    # Instanciation des policiers
    policier_id = kesaio["Policier" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((policier_id, RDF.type, kesaio["Policier"]))
    g.add((policier_id, hasRole, kesaio["encadrementEtapplication"]))  # Assurez-vous que le rôle est correct.
    g.add((policier_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((policier_id, common_core_ns["is_affiliated_with"], kesaio["policeNationale"]))
    g.add((policier_id, participatesIn, uri_actDeRamassage))
for i in range(0,nombre_officiers_samu): 
    # Instanciation des officiers du SAMU
    officier_samu_id = kesaio["OfficierRamassage" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((officier_samu_id, RDF.type, kesaio["Membre_SAMU"]))
    g.add((officier_samu_id, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((officier_samu_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((officier_samu_id, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((officier_samu_id, participatesIn, uri_actDeRamassage))

In [99]:
# Écrire et exécuter une requête SPARQL pour obtenir toutes les instances de la classe "Victime"
query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type :Victime.
    }
"""
results = g.query(query)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((uri_actDeRamassage, kesaio["ramasser"], victime_instance))
    g.add((victime_instance, kesaio["ramassé"], Literal(True, datatype=XSD.boolean)))

# Le poste médical avancé (PMA)

In [100]:
latPMA = float(latPRV) + 1000
lonPMA = float(lonPRV) + 1000

uri_pma = kesaio["Poste_médical_avancé" + "_1"]
g.add((uri_pma, RDF.type, kesaio["Poste_médical_avancé"]))
g.add((uri_pma, kesaio["longitude"], Literal(lonPMA, datatype=XSD.decimal)))
g.add((uri_pma, kesaio["latitude"], Literal(latPMA, datatype=XSD.decimal)))
g.add((uri_pma, kesaio["nombreDeLit"], Literal(50, datatype=XSD.integer)))

uri_rolePMA = kesaio["officierPMA"]

for i in range(0, 3): 
    identifiant_unique = uuid.uuid4()
    uri_OPMA = kesaio["OfficierPMA" + str(i)]  
    g.add((uri_OPMA, RDF.type, kesaio["Membre_SAMU"])) 
    g.add((uri_OPMA, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((uri_OPMA, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((uri_OPMA, hasRole, uri_rolePMA))
    g.add((uri_rolePMA, kesaio["estSuperviséPAr"], kesaio["commandantSAMU"]))
    g.add((uri_OPMA, kesaio["monter"], uri_pma))
    # g.add((uri_pma, kesaio["installed_by"], uri_rolePMA))

### Acte de secours des victimes

In [101]:
uri_actDeSoins = kesaio["Acte_de_secours_des_victimes" + "_1"]
g.add((uri_actDeSoins, RDF.type, kesaio["Acte_de_secours_des_victimes"]))
g.add((uri_actDeSoins, common_core_ns["occurs_at"], uri_pma))
g.add((uri_op, common_core_ns["has_process_part"], uri_actDeSoins))
g.add((uri_actDeSoins, kesaio["needs"], kesaio["equipeMédicale"]))
g.add((uri_actDeSoins, kesaio["needs"], kesaio["vehiculeDeSecoursEtDassistanceAuxVictimes"]))
g.add((uri_actDeSoins, kesaio["needs"], kesaio["vehiculeDeSoutienSanitaire"]))

nombreBlesse

10

In [102]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?blessesASoinger) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Etat_victime.
        ?Etat_victime cco:designated_by ?Code_etat.
        FILTER (?Code_etat = kesaio:ur)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/",
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

results = g.query(requete)

# Parcourir les résultats et afficher les victimes
for row in results:
    blessesASoinger = row.blessesASoinger

blessesASoinger = int(blessesASoinger)
print(blessesASoinger)

4


In [103]:
# Calculez combien de médecins et d'infirmiers sont nécessaires
nombre_de_medecins = blessesASoinger // 5
nombre_d_infirmiers = blessesASoinger // 5

# Appliquez la règle
if blessesASoinger % 5 != 0:
    nombre_de_medecins += 1
    nombre_d_infirmiers += 1

# Affichez les résultats
print("Nombre total de victimes :", blessesASoinger)
print("Nombre de médecins nécessaires :", nombre_de_medecins)
print("Nombre d'infirmiers nécessaires :", nombre_d_infirmiers)

Nombre total de victimes : 4
Nombre de médecins nécessaires : 1
Nombre d'infirmiers nécessaires : 1


In [104]:
for i in range(0,nombre_de_medecins): 
    # Instanciation des officiers du SAMU
    medecin = kesaio["Médecin_urgentiste_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((medecin, RDF.type, kesaio["Membre_SAMU"]))
    g.add((medecin, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((medecin, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((medecin, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((medecin, participatesIn, uri_actDeSoins))

for i in range(0,nombre_d_infirmiers): 
    # Instanciation des officiers du SAMU
    infirmier = kesaio["Infirmier_SAMU_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((infirmier, RDF.type, kesaio["Membre_SAMU"]))
    g.add((infirmier, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((infirmier, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((infirmier, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((infirmier, participatesIn, uri_actDeSoins))

In [105]:
query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Etat_victime.
        ?Etat_victime cco:designated_by ?Code_etat.
        FILTER (?Code_etat = kesaio:ur)
    }
"""
# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/",
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((uri_actDeSoins, kesaio["soigner"], victime_instance))
    g.add((victime_instance, kesaio["soigné"], Literal(True, datatype=XSD.boolean)))

##### Suivi psycho medical

In [106]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?suivi_psycho_medical) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Etat_victime.
        ?Etat_victime cco:designated_by ?Code_etat.
        FILTER (?Code_etat = kesaio:ump)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/",
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

results = g.query(requete)

# Parcourir les résultats et afficher les victimes
for row in results:
    suivi_psycho_medical = row.suivi_psycho_medical

suivi_psycho_medical = int(suivi_psycho_medical)
print(suivi_psycho_medical)

2


In [107]:
nombre_psychologues = suivi_psycho_medical // 5

if suivi_psycho_medical % 5 != 0:
    nombre_psychologues += 1

for i in range(0,nombre_psychologues): 
    # Instanciation des officiers du SAMU
    infirmier = kesaio["Psychologue_SAMU_" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((infirmier, RDF.type, kesaio["Membre_SAMU"]))
    g.add((infirmier, hasRole, uri_offPRV))  # Assurez-vous que le rôle est correct.
    g.add((infirmier, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((infirmier, common_core_ns["is_affiliated_with"], kesaio["samu"]))
    g.add((infirmier, participatesIn, uri_actDeSoins))

query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Etat_victime.
        ?Etat_victime cco:designated_by ?Code_etat.
        FILTER (?Code_etat = kesaio:ump)
    }
"""
# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/",
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((uri_actDeSoins, kesaio["soigner"], victime_instance))
    g.add((victime_instance, kesaio["soigné"], Literal(True, datatype=XSD.boolean)))

##### Gestion des morts

In [108]:
nombre_pompiers = nombreMorts // 5

for i in range(0,nombre_pompiers): 
    pompier_id = kesaio["Pompier" + str(i)]
    identifiant_unique = uuid.uuid4()
    g.add((pompier_id, RDF.type, kesaio["Pompier"])) 
    g.add((pompier_id, hasRole, kesaio["equipierMedical"]))
    g.add((pompier_id, kesaio["identifiant"], Literal(identifiant_unique, datatype=XSD.string)))
    g.add((pompier_id, common_core_ns["is_affiliated_with"], kesaio["sapeurPompier"]))
    g.add((pompier_id, participatesIn, uri_actDeSoins))


query = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Etat_victime.
        ?Etat_victime cco:designated_by ?Code_etat.
        FILTER (?Code_etat = kesaio:décédé)
    }
"""
# Préparez la requête SPARQL
requete = prepareQuery(query, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/",
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((victime_instance, kesaio["deposerAlaMorgue"], Literal(True, datatype=XSD.boolean)))

##### Gestion des rescapés

In [109]:
requete_sparql = """
    SELECT ?victime
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Etat_victime.
        ?Etat_victime cco:designated_by ?Code_etat.
        FILTER (?Code_etat = kesaio:indemne)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/",
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

# Parcourir les résultats et afficher les victimes
results = g.query(requete)
# Parcourir les résultats et obtenir les instances de la classe "Victime"
for row in results:
    victime_instance = row.victime
    g.add((victime_instance, kesaio["relogé"], Literal(True, datatype=XSD.boolean)))

### Acte d'évacuation des victimes

In [110]:
requete_sparql = """
    SELECT (COUNT(?victime) as ?blessesAEvacuer) 
    WHERE {
        ?victime rdf:type kesaio:Victime.
        ?victime kesaio:has_status ?Etat_victime.
        ?Etat_victime cco:designated_by ?Code_etat.
        FILTER (?Code_etat = kesaio:ua)
    }
"""

# Préparez la requête SPARQL
requete = prepareQuery(requete_sparql, initNs={"kesaio": "http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/",
                                                "cco": "http://www.ontologyrepository.com/CommonCoreOntologies/"})

results = g.query(requete)

# Parcourir les résultats et afficher les victimes
for row in results:
    blessesAEvacuer = row.blessesAEvacuer

blessesAEvacuer = int(blessesAEvacuer)
print(blessesAEvacuer)

1


In [111]:
# Enregistrez l'ontologie enrichie dans un fichier RDF (si nécessaire)
g.serialize("ontologie_enrichie.rdf", format="xml")

# Vous pouvez également imprimer le contenu de l'ontologie
for triple in g:
    print(triple)

(rdflib.term.BNode('Ndd0b344328af43b08069f7bafdde83bc'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#rest'), rdflib.term.BNode('N6be59d8e1ab4496da9cf459e90088031'))
(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Camion_de_pompiers'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
(rdflib.term.BNode('Na3978c1088e841a9beaa43f60b7d62e7'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'), rdflib.term.URIRef('http://www.ontologyrepository.com/CommonCoreOntologies/ActOfTargeting'))
(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Gendarmerie_des_Transports_Aériens'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.Literal('Gendarmerie des Transports Aériens'))
(rdflib.term.URIRef('http://www.semanticweb.org/mandiaye/ontologies/2023/7/CatastropheV2/Acte_de_rama